# RoseLoRA: Row and Column-wise Sparse Low-rank Adaptation

This notebook allows you to run the official implementation of RoseLoRA from the EMNLP'24 paper: [RoseLoRA: Row and Column-wise Sparse Low-rank Adaptation of Pre-trained Language Model for Knowledge Editing and Fine-tuning](https://arxiv.org/abs/2406.10777)


In [1]:
!nvidia-smi

Fri Apr 11 11:55:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/ayushnangia/roselora
%cd roselora

Cloning into 'roselora'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 144 (delta 26), reused 124 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 14.69 MiB | 3.90 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Updating files: 100% (98/98), done.
/content/roselora


In [3]:
!pip install  -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import os

# Parameters that can be adjusted
data_size = -1  # How many samples to test with. Set to -1 to use all samples
sequential_edit = 1  # Sequential editing length
retrain = 0  # Whether to enforce retrain. Set to 0 to use existing results if available
base_model = "llama2"  # LLM to edit. Currently only supports "llama2"

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print(f"Parameters:\n"
      f"- Data size: {data_size}\n"
      f"- Sequential edit: {sequential_edit}\n"
      f"- Retrain: {retrain}\n"
      f"- Base model: {base_model}")

Parameters:
- Data size: -1
- Sequential edit: 1
- Retrain: 0
- Base model: llama2


In [ ]:
from huggingface_hub import login

# Enter your HuggingFace token to access LLaMA-2 model
# You can get this token from https://huggingface.co/settings/tokens
from google.colab import userdata
hf_token = input("Enter your HuggingFace token to access LLaMA-2 model: ")
login(token=userdata.get('HF_TOKEN'))

In [4]:
import sys
sys.path.append('.')
!cd ./roselora/examples && python run_zsre.py \
    --editing_method RoseLoRA \
    --ds_size {data_size} \
    --sequential_edit {sequential_edit} \
    --retrain {retrain} \
    --data_dir=/content/roselora/examples/data/ZsRE \
    --hparams_dir=../hparams/RoseLoRA/llama-7b \
    --base_model={base_model}

/usr/local/lib/python3.11/dist-packages/timm/models/hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
2025-04-11 12:09:31.208060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744373371.440110    4276 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744373371.504117    4276 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 12:09:31.996070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical ope

In [ ]:
# List output files
!ls -la ./roselora/examples/output/RoseLoRA

In [ ]:
import json
import matplotlib.pyplot as plt
import os
import numpy as np

# Find the most recent results file
output_dir = "./roselora/examples/output/RoseLoRA"
if os.path.exists(output_dir):
    result_files = [f for f in os.listdir(output_dir) if f.endswith(".json") and not f.startswith(".")]
    if result_files:
        latest_file = sorted(result_files)[-1]
        result_path = os.path.join(output_dir, latest_file)
        print(f"Analyzing results from: {result_path}")

        with open(result_path, 'r') as f:
            results = json.load(f)

        # Extract metrics
        edit_success = [data['post']['rewrite_acc'][0] for data in results]
        rephrase_success = [data['post']['rephrase_acc'][0] for data in results]

        # Calculate averages
        avg_edit_success = sum(edit_success) / len(edit_success) * 100
        avg_rephrase_success = sum(rephrase_success) / len(rephrase_success) * 100

        # Extract portability and locality metrics
        portability_values = []
        locality_values = []

        for data in results:
            # Portability
            port_case_list = []
            for key in data['post']['portability'].keys():
                port_vals = data['post']['portability'][key]
                if port_vals:
                    port_case_list.append(sum(port_vals) / len(port_vals) * 100)
            if port_case_list:
                portability_values.append(np.mean(port_case_list))

            # Locality
            loc_case_list = []
            for key in data['post']['locality'].keys():
                loc_vals = data['post']['locality'][key]
                if loc_vals:
                    loc_case_list.append(sum(loc_vals) / len(loc_vals) * 100)
            if loc_case_list:
                locality_values.append(np.mean(loc_case_list))

        # Calculate averages
        avg_portability = np.mean(portability_values) if portability_values else 0
        avg_locality = np.mean(locality_values) if locality_values else 0

        # Plot results
        metrics = ['Edit Success', 'Rephrase Success', 'Portability', 'Locality']
        values = [avg_edit_success, avg_rephrase_success, avg_portability, avg_locality]

        plt.figure(figsize=(10, 6))
        plt.bar(metrics, values, color=['blue', 'green', 'red', 'purple'])
        plt.ylabel('Percentage (%)')
        plt.title('RoseLoRA Performance Metrics')
        plt.ylim(0, 100)

        # Add value labels on bars
        for i, v in enumerate(values):
            plt.text(i, v + 2, f"{v:.2f}%", ha='center')

        plt.show()

        print(f"Edit Success: {avg_edit_success:.2f}%")
        print(f"Rephrase Success: {avg_rephrase_success:.2f}%")
        print(f"Portability: {avg_portability:.2f}%")
        print(f"Locality: {avg_locality:.2f}%")
    else:
        print("No result files found.")
else:
    print(f"Output directory {output_dir} not found.")

In [ ]:
# Display current hyperparameters
!cat hparams/RoseLoRA/llama-7b.yaml

In [ ]:
# Function to modify sparsity parameter
import fileinput
import sys

def modify_sparsity(new_sparsity=0.05):
    file_path = 'easyeditor/models/roselora/roselora_main.py'
    for line in fileinput.input(file_path, inplace=True):
        if 'sparsity = ' in line and not line.strip().startswith('#'):
            sys.stdout.write(f"    sparsity = {new_sparsity}  # Modified from original value\n")
        else:
            sys.stdout.write(line)
    print(f"Changed sparsity to {new_sparsity}")

# Uncomment the next line to modify sparsity
# modify_sparsity(0.1)  # Change to your desired value